# Xenon XML Repair Library Tutorial

This notebook provides an interactive tutorial for the Xenon XML repair library, designed specifically for handling malformed XML from LLMs.

## Installation

```bash
# Install from GitHub (package not yet on PyPI)
pip install git+https://github.com/MarsZDF/xenon.git
```

In [ ]:
# Import Xenon
from xenon import repair_xml, parse_xml, repair_xml_safe, repair_xml_with_report

## 1. Basic Repair - Truncated XML

LLMs often generate truncated XML when hitting token limits.

In [ ]:
truncated_xml = '<root><user name="Alice"><age>30'
print(f"Input:  {truncated_xml}")

repaired = repair_xml(truncated_xml)
print(f"Output: {repaired}")

## 2. Extracting XML from Conversational Text

LLMs often add conversational fluff around the XML.

In [ ]:
llm_response = "Sure! Here is the data: <root><item>data</item></root> Hope this helps!"
print(f"Input:  {llm_response}")

repaired = repair_xml(llm_response)
print(f"Output: {repaired}")

## 3. Fixing Malformed Attributes

LLMs sometimes forget to quote attribute values.

In [ ]:
malformed = "<user name=John age=30>Hello</user>"
print(f"Input:  {malformed}")

repaired = repair_xml(malformed)
print(f"Output: {repaired}")

## 4. Escaping Unescaped Entities

Special characters like `&`, `<`, `>` need to be escaped in XML.

In [ ]:
unescaped = "<message>Tom & Jerry < > test</message>"
print(f"Input:  {unescaped}")

repaired = repair_xml(unescaped)
print(f"Output: {repaired}")

## 5. Parsing XML to Dictionary

Convert XML to Python dictionaries for easy data extraction.

In [ ]:
xml = '<user name="Alice"><age>30</age><city>NYC</city></user>'
print(f"Input XML: {xml}")

parsed = parse_xml(xml)
print(f"\nParsed Dictionary:")
import json

print(json.dumps(parsed, indent=2))

## 6. Repair Reporting

Get detailed reports about what was fixed.

In [ ]:
xml = "<root><user name=Alice>Tom & Jerry<item>unclosed"
print(f"Input: {xml}\n")

repaired, report = repair_xml_with_report(xml)
print(f"Output: {repaired}\n")
print(report.summary())
print(f"\nTotal repairs: {len(report)}")

## 7. Security Features

Protect against XSS and XXE attacks in untrusted XML.

In [ ]:
dangerous = '<?php echo "test"; ?><script>alert("xss")</script><data>content</data>'
print(f"Input: {dangerous}\n")

safe = repair_xml_safe(dangerous, strip_dangerous_pis=True, strip_dangerous_tags=True)
print(f"Sanitized: {safe}")

## 8. Batch Processing

Process multiple XML strings efficiently.

In [ ]:
from xenon.utils import batch_repair

xml_batch = [
    "<root><item>first",
    "<root><item>second",
    "<root><item>third",
]

results = batch_repair(xml_batch)

for i, (repaired, error) in enumerate(results):
    if error is None:
        print(f"✓ Item {i + 1}: {repaired}")
    else:
        print(f"✗ Item {i + 1}: Error - {error}")

## 9. Advanced Configuration

Use configuration objects for complex setups.

In [ ]:
from xenon import XMLRepairEngine, XMLRepairConfig, SecurityFlags, RepairFlags

config = XMLRepairConfig(
    security=SecurityFlags.STRIP_DANGEROUS_PIS | SecurityFlags.STRIP_EXTERNAL_ENTITIES,
    repair=RepairFlags.SANITIZE_INVALID_TAGS | RepairFlags.WRAP_MULTIPLE_ROOTS,
)

engine = XMLRepairEngine(config)

test = "<123invalid>test</123invalid><another>root</another>"
print(f"Input:  {test}")

result = engine.repair_xml(test)
print(f"Output: {result}")

## 10. Real-World Example: Processing LLM Output

A complete example of processing XML from an LLM API.

In [ ]:
# Simulated LLM response (replace with actual API call)
llm_output = """
Here's the extracted data in XML format:

<customers>
    <customer id=1>
        <name>Acme Corp</name>
        <email>contact@acme.com</email>
        <orders>
            <order id="A123" total="1500.00"/>
            <order id="A124" total="2300
"""

print("Processing LLM output...\n")

# Repair and parse
repaired, report = repair_xml_with_report(llm_output)
data = parse_xml(repaired)

# Display results
print("Repairs performed:")
print(report.summary())
print("\nExtracted data:")
print(json.dumps(data, indent=2))

## Summary

Xenon provides:

- ✅ Automatic truncation repair
- ✅ Conversational text extraction
- ✅ Malformed attribute fixing
- ✅ Entity escaping
- ✅ XML to dictionary parsing
- ✅ Detailed repair reporting
- ✅ Security features (XSS/XXE prevention)
- ✅ Batch processing
- ✅ Advanced configuration options

Perfect for making LLM-generated XML reliable in production!